In [2]:
import sqlite3
import pandas as pd
import json

In [41]:
conn = sqlite3.connect('both.db') 
c = conn.cursor()

#need distinct primary key for each entry
c.execute('''
          CREATE TABLE IF NOT EXISTS Both
          ([key] TEXT PRIMARY KEY, [commodity] STRING, [period] STRING, [pos] TEXT, [word_num] TEXT,
          [value] TEXT)
          ''')

In [ ]:
'''
period | pos | word# | value
-------------------------------
   1   |  j  |   1   |  great
-------------------------------
   1   |  n  |   2   |  filthy

'''

In [46]:
path = '/Users/rossrichesin/Desktop/dataPlus/opiumPeriods.json'
f = open(path,'r')
data = f.read()
period_dict = json.loads(data)

In [45]:
len(period_dict['pos'])

397

In [13]:
df = pd.DataFrame(period_dict)

In [14]:
df 

,period,pos,word_num,value
0,1,j,1,white
1,1,j,2,black
2,1,j,3,great
3,1,j,4,good
4,1,j,5,half
...,...,...,...,...
259,5,v,6,find
260,5,v,7,eat
261,5,v,8,thicken
262,5,v,9,break


In [20]:
df.loc[df['pos']=='p'].info()
#each should have 50 for opium
#and 70 for tobacco

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 200 to 213
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   period    14 non-null     int64 
 1   pos       14 non-null     object
 2   word_num  14 non-null     int64 
 3   value     14 non-null     object
dtypes: int64(2), object(2)
memory usage: 560.0+ bytes


In [42]:
conn = sqlite3.connect('both.db') 
c = conn.cursor()

paths = ['/Users/rossrichesin/Desktop/dataPlus/json/tobaccoPeriods.json', '/Users/rossrichesin/Desktop/dataPlus/json/opiumPeriods.json']
commodities = ['t','o'] #commodities (tobacco, opium)
for commodity, path in zip(commodities,paths):
      f = open(path,'r')
      data = f.read()
      period_dict = json.loads(data)
      keys = [commodity+str(x) for x in range(max([len(period_dict[i]) for i in period_dict.keys()]))]
      for key, period,pos,word_num, value in zip(keys,period_dict['period'],period_dict['pos'],period_dict['word_num'],period_dict['value']):
            statement = "INSERT OR IGNORE INTO Both (key,commodity,period, pos, word_num, value) VALUES (?,?,?,?,?,?);"
            data = (key,commodity, period,pos,word_num,value)
            c.execute(statement, data)      
            conn.commit()


In [43]:
conn = sqlite3.connect('both.db') 
c = conn.cursor()
c.execute('''
          SELECT * FROM Both
          ''')

df = pd.DataFrame(c.fetchall(), columns=['key','commodity','period', 'pos', 'word_num', 'value'])
df

,key,commodity,period,pos,word_num,value
0,t0,t,5,j,1,good
1,t1,t,5,j,2,great
2,t2,t,5,j,3,foreign
3,t3,t,5,j,4,english
4,t4,t,5,j,5,diverse
...,...,...,...,...,...,...
656,o259,o,5,v,6,find
657,o260,o,5,v,7,eat
658,o261,o,5,v,8,thicken
659,o262,o,5,v,9,break
